In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [2]:
base_dir = r"D:\DATA\Training"
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

In [3]:
# Load images and labels
import os
data = []
labels = []

for category in categories:
    path = os.path.join(base_dir, category)
    class_num = categories.index(category)
    if not os.path.exists(path):
        print(f"Directory {path} does not exist!")
        continue
    class_num = categories.index(category)
    for img in os.listdir(path):
        try:
            img_array = load_img(os.path.join(path, img), target_size=(64, 64))  # Resize images
            img_array = img_to_array(img_array)
            data.append(img_array)
            labels.append(class_num)
        except Exception as e:
            print(f"Error loading image: {e}")

# Convert lists to numpy arrays
data = np.array(data, dtype='float32')
labels = np.array(labels)


In [4]:
label_counts=np.bincount(labels)
for i, category in enumerate(categories):
    print(f"Number of images in category '{category}': {label_counts[i]}")

Number of images in category 'glioma': 1321
Number of images in category 'meningioma': 1339
Number of images in category 'notumor': 1595
Number of images in category 'pituitary': 1457


In [5]:
# Normalize data
data /= 255.0
from sklearn.model_selection import StratifiedShuffleSplit

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [6]:
# Flatten the images for PCA
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

In [7]:
# Standardize the data
scaler = StandardScaler()
X_train_flat = scaler.fit_transform(X_train_flat)
X_test_flat = scaler.transform(X_test_flat)

In [8]:
# Apply PCA
pca = PCA(n_components=50)  # Number of principal components to keep
X_train_pca = pca.fit_transform(X_train_flat)
X_test_pca = pca.transform(X_test_flat)

In [9]:
# Train SVM model
svm = SVC(kernel='linear', C=1)
svm.fit(X_train_pca, y_train)

SVC(C=1, kernel='linear')

In [10]:
# Predict on test data
y_pred = svm.predict(X_test_pca)

In [11]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define unique class names for evaluation
unique_classes = np.unique(y_test)
unique_classes_names = [categories[i] for i in unique_classes]


In [12]:
from sklearn.ensemble import RandomForestClassifier

# Train a Random Forest model
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train_pca, y_train)

# Predict on test data
y_pred_rfc = rfc.predict(X_test_pca)

# Evaluate the Random Forest model
print("\nRandom Forest Classification Report:")
print(classification_report(y_test, y_pred_rfc, target_names=unique_classes_names))
print("Random Forest Accuracy Score:")
print(accuracy_score(y_test, y_pred_rfc))




Random Forest Classification Report:
              precision    recall  f1-score   support

      glioma       0.91      0.84      0.88       288
  meningioma       0.82      0.81      0.82       265
     notumor       0.95      0.98      0.97       291
   pituitary       0.92      0.97      0.94       299

    accuracy                           0.90      1143
   macro avg       0.90      0.90      0.90      1143
weighted avg       0.90      0.90      0.90      1143

Random Forest Accuracy Score:
0.9037620297462817


In [13]:
from sklearn.neighbors import KNeighborsClassifier

# Train a K-Nearest Neighbors model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_pca, y_train)

# Predict on test data
y_pred_knn = knn.predict(X_test_pca)

# Evaluate the KNN model
print("\nK-Nearest Neighbors Classification Report:")
print(classification_report(y_test, y_pred_knn, target_names=unique_classes_names))
print("K-Nearest Neighbors Accuracy Score:")
print(accuracy_score(y_test, y_pred_knn))



K-Nearest Neighbors Classification Report:
              precision    recall  f1-score   support

      glioma       0.86      0.85      0.86       288
  meningioma       0.82      0.78      0.80       265
     notumor       0.94      0.94      0.94       291
   pituitary       0.92      0.97      0.94       299

    accuracy                           0.89      1143
   macro avg       0.89      0.89      0.89      1143
weighted avg       0.89      0.89      0.89      1143

K-Nearest Neighbors Accuracy Score:
0.8888888888888888


In [14]:
print("\nModel Comparison:")
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rfc):.2f}")
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")



Model Comparison:
SVM Accuracy: 0.81
Random Forest Accuracy: 0.90
KNN Accuracy: 0.89


In [15]:
unique_classes=np.unique(y_test)
unique_classes_names=[categories[i] for i in unique_classes]

In [16]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img

def preprocess_image(image_path, target_size=(64, 64)):
    """Load and preprocess a single image."""
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize
    img_array = img_array.reshape(1, -1)  # Flatten and reshape for the model
    img_array = scaler.transform(img_array)  # Apply the same scaler used during training
    img_array = pca.transform(img_array)  # Apply PCA transformation
    return img_array

def predict_image(model, image_array, categories):
    """Predict the class of a single image."""
    prediction = model.predict(image_array)
    class_idx = prediction[0]
    return categories[class_idx]

# Path to the image you want to test
test_image_path = "D:\\DATA\\Training\\meningioma\\Tr-me_0013.jpg"
# Preprocess the image
test_image_array = preprocess_image(test_image_path)

# Predict the class of the image
predicted_class = predict_image(rfc, test_image_array, categories)

print(f"The predicted class for the test image is: {predicted_class}")

The predicted class for the test image is: meningioma
